The goal of this notebook is to derive the $Q$ matrix (aka. Infinetestimal generator Matrix) from data generated by a gillespie simulated CTMC model of a SIR disease process. 

To begin, lets define the disease model.

The SIR compartmental model features 3 compartiments (Susceptible, Infectious, and Recovered) though which N individuals pass between (i.e. $S + I + R = N$) over time. To impliment this, lets define a type to represent the state of an SIR system at time t, $X(t) = \{S, I, R\}$.


In [ ]:
struct SIR_state
    S::Int
    I::Int
    R::Int
end

```{dot}
digraph SIR {
    rankdir = "LR"
    S->I
    I->R
}
```


 In an SIR system there are two possible events that can occur after a certain time $\Delta t$: infection (i.e. an individual moves from the Susceptible compartment to the Infectious compartment) $X(t + \Delta t) = \{s-1, i + 1, r\}$ and  recovery $X(t + \Delta t) = \{s, i-1, r +1\}$. Lets define these as triples which we can add to our SIR_State variable


In [ ]:
SIR_events = (
    infection = [-1, 1, 0],
    recovery = [0,-1,1]
)

Only one event can occur at a time* which event will occur? Epidemiologcal theory tells us that, assuming homogenous mixing of infectious and susceptible individuals there will be $S \times I$ opportunities for contact. Only a proportion of these opporunities will result in infection, so let the parameter $\beta$, the 'transmission rate' l modulate this. 

Simiularly, we'll specify a constant parameter, \lambda, which will modulate the rate of recovery.
Lets define a 'transition' function which, given a state $X(t) = \{s, i,r\}$ will return the ___ of each event occurring


In [ ]:
function Transition_Function(X, β, γ)
  (S,I,R) = X
  infection = β*S*I
  recovery = γ*I
  [infection,recovery]
end;

Ok thats enough setting up, lets simulate the evolution of the system throrugh time.


To record the output, I'll create a dataframe for the state variables and the time


In [ ]:
using DataFrames

SIR_CTMC = DataFrame(
    t = Vector{Float64}(),
    S = Vector{Int64}(),
    I = Vector{Int64}(),
    R = Vector{Int64}())

and set the initial conditions


In [ ]:
X0 = [99,1,0]
t0 = 0
β = 0.005
γ = 0.1
Xt = X0
t = t0
push!(SIR_CTMC, [t;Xt])

now for a time step. Delta t will have an exponential distribution with parameter 1/\sum{a+b}


In [ ]:
using Distributions

Δt = rand(Exponential(1/sum(Transition_Function(Xt,β, γ))))
t += Δt

Next we need to determine which event will happen. The probability of a transition will be given by the proportion of the 


In [ ]:
Pt = Transition_Function(Xt,β, γ)
Pi = Pt[1]/sum(Pt)
Pr = Pt[2]/sum(Pt)

To sample one event with probability weighting according to its relative propensity,

In [ ]:
t = rand() 
i = 1
cw = Pt[1]
while cw < t && i < length(Pt)
    i += 1
    @inbounds cw += Pt[i]
end

Δx = SIR_events[i]

Xt += Δx

push!(SIR_CTMC, [t;Xt])

We probably want to automate that step, so lets put that in a function


In [ ]:
tend = 50
while t < tend
    Δt = rand(Exponential(1/sum(Transition_Function(Xt, β, γ))))


    t += Δt

    Pt = Transition_Function(Xt,β, γ)
    Ptn = Pt ./ sum(Pt)

    i = 1
    cw = Ptn[1]
    while cw < rand() && i < length(Ptn)
        i += 1
        @inbounds cw += Ptn[i]
    end

    Δx = SIR_events[i]

    Xt += Δx

    push!(SIR_CTMC, [t;Xt])
end

lets plot the results for a clearer picture


In [ ]:
using StatsPlots

@df SIR_CTMC plot(:t, [:S, :I, :R] )

The probability that a susceptible individual will become infected $p_{s-1, i+1}\leftarrow{s, i} = \frac$



Since $R = S+I -N$, se can represent the state of the system with $\{S, I\}$, where $S \in \{0,1, ... , N-I-R\}$, $I \in \{0,1, ... , N-S-R\}$.

for example, when N = 3, we can represent the system with a graph:

As a discrete time process this Markov chain can be represented by the transition matrix 
$$
p = \begin{bmatrix}
\end{bmatrix}
$$


looks reasonable. How about in comparison to the deterministic ODE solution


In [ ]:
using DifferentialEquations

function SIRModel!(du, u, p, t)
    S, I, R = u
    β, γ = p
    du[1] = -β*S*I
    du[2] = β*S*I-γ*I
    du[3] = γ*I
end 

p = [.5,.1]
u0 = [.99,.01,0]
tspan = (0.0, 50)

prob = ODEProblem(SIRModel!, u0, tspan, p)

sol = solve(prob)

plot(sol)

They're not identical but maybe this is to be expected given the small sample size. We could inrease the sample size (which extended to \infty would equal the ODE), but lets instead perform the N = 100 simulation a few more times.

For convenience I'll wrap the whole simulation in a function which takes initial conditions and parameters in and outputs the data


In [ ]:
function SIR_sim(X0::Vector{Int64}, tmax::Float64, params::Vector{Float64} )

    SIR_events = (
        infection = [-1, 1, 0],
        recovery = [0,-1,1]
    )

    function Transition_Function(X, β, γ)
    (S,I,R) = X
    infection = β*S*I
    recovery = γ*I
    [infection,recovery]
    end

    SIR_CTMC = DataFrame(
    t = Vector{Float64}(),
    S = Vector{Int64}(),
    I = Vector{Int64}(),
    R = Vector{Int64}())


    X0 = X0
    t0 = 0
    β = params[1]
    γ = params[2]
    Xt = X0
    t = t0
    push!(SIR_CTMC, [t;Xt])

    Δt = rand(Exponential(1/sum(Transition_Function(Xt,β, γ))))
    t += Δt
    tmax = tmax
    while t < tmax
        Δt = rand(Exponential(1/sum(Transition_Function(Xt, β, γ))))


        t += Δt

        Pt = Transition_Function(Xt,β, γ)
        Ptn = Pt ./ sum(Pt)

        i = 1
        cw = Ptn[1]
        while cw < rand() && i < length(Ptn)
            i += 1
            @inbounds cw += Ptn[i]
        end

        Δx = SIR_events[i]

        Xt += Δx

        push!(SIR_CTMC, [t;Xt])
   

    end
     return SIR_CTMC
end

and run this 50 times


In [ ]:
Sims = DataFrame(
    rep = Vector{Int64}(), 
    t = Vector{Float64}(),
    S = Vector{Int64}(),
    I = Vector{Int64}(),
    R = Vector{Int64}()) 

for i in 1:50
    sim = SIR_sim([99,1,0], 50.0, [.005, .1])
    newrep =  hcat(
        sim, 
        DataFrame(rep = fill(i, nrow(sim))
        ))
    push!(Sims, vcat(Sims, newrep))
end


@df df plot(:t, [:S, :I, :R])

In continuous time, siple transition probabilities are not sufficient to describe the process. We need the Transition rates which are related to the trasition probabilities
$$p _ { i j } = \frac { q _ { i j } } { \sum _ { k } q _ { i k } } = \frac { q _ { i j } } { q _ { i } }$$
-----
Gillespie algorithm

1. Define event space (Susceptible -> infectious, Infectious -> Recovered) and the propensity function for each event 
    $p\_{inf} = \beta \times S \times I$
    $p\_{rec} = \gamma \times I$

2. Determine the time until the next event ($\tau$) by sampling from an exponential distribution with parameter $\frac{1}{\sum{p}}$

3. Determine which event occurs at time $t + \tau$ by weighting a random sample of $x\sim unif(0,1)$ according top each event's propensity and taking the large of the two. 

4. Update time and state to be t + \tau and ___ 